In [44]:
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

In [45]:
# Load the OpenAI API key from the .env file
load_dotenv()
api_key = os.getenv("api_key")
if api_key is None:
    raise ValueError("The api_key environment variable is not set.")

In [46]:
from auxiliar import add_messages

In [47]:
class SyntheticUserMessage(BaseModel):

    message: str = Field(
        ...,
        title="Message",
        description="The user message to generate for the target task intention.",
    )


class ListSyntheticUserMessages(BaseModel):

    messages: list[SyntheticUserMessage] = Field(
        ...,
        title="Messages",
        description="The list of synthetic user messages to generate for the target task intention.",
    )

output_parser = PydanticOutputParser(pydantic_object=ListSyntheticUserMessages)

In [48]:
system_prompt = """
You are tasked with generating synthetic user messages for an Fantasy Football platform called Ball IQ, which specializes in premier league football.
The user intentions are:
{user_intentions}

Your task is to create {k} distinct messages for the following target task intention:
{target_task_intention}

Specific information about the target task intention:
{target_task_intention_description}

Follow these guidelines:
1. Focus exclusively on the target task intention, ensuring the message is relevant.
2. Each message should be between 7 and 25 words.
3. Avoid including any details or references to other user intentions.
4. Ensure the messages sound natural and typical of user queries for the given intention.
5. Follow the provided format strictly to maintain consistency.
7. Make sure most of the message are words and not nouns

Message format:
{format_instructions}
"""

prompt = PromptTemplate(
    template=system_prompt,
    input_variables=["k", "user_intentions", "target_task_intention" "target_task_intention_description", "format_instructions"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

In [49]:
llm = ChatOpenAI(api_key= api_key, temperature=0.0, model="gpt-4o-mini")

user_intentions = ["check_upcoming_fixtures", "optimize_lineup",  "putting_players_in_starting11", "know_information_about_stats", "recommend_players", "transfer_players", "view_or_update_team_points", "view_players_stats", "chit_chat_about_company"]
k = 50 # Number of synthetic user messages to generate for each target task intention

file_name = "synthetic_intetions.json"

synthetic_data_chain = prompt | llm | output_parser

# Intention 1: Check upcoming fixtures

In [50]:
intention = "check_upcoming_fixtures"

description = "The user wants to know the upcoming matches for a player in their team. To do so they ask for a player and the week they want information.\
                They might ask questions related what the next game of the player will be or who a player will play against in a certain week."

response = synthetic_data_chain.invoke({"k": k, "user_intentions": user_intentions, "target_task_intention": intention, "target_task_intention_description": description})

check_upcoming_fixtures_messages = []
for message in response.messages:
    check_upcoming_fixtures_messages.append({"Intention":intention, "Message":message.message})

In [51]:
check_upcoming_fixtures_messages[:5]

[{'Intention': 'check_upcoming_fixtures',
  'Message': 'What are the upcoming fixtures for Harry Kane this week?'},
 {'Intention': 'check_upcoming_fixtures',
  'Message': 'Can you tell me who Mohamed Salah is playing against next week?'},
 {'Intention': 'check_upcoming_fixtures',
  'Message': "When is Son Heung-min's next match scheduled?"},
 {'Intention': 'check_upcoming_fixtures',
  'Message': 'What are the fixtures for Kevin De Bruyne in the upcoming week?'},
 {'Intention': 'check_upcoming_fixtures',
  'Message': 'Who does Bruno Fernandes face in the next game?'}]

In [52]:
add_messages(check_upcoming_fixtures_messages, file_name)

# Intention 2: Optimize Lineup

In [53]:
intention = "optimize_lineup"

description = "The user wants to optimize their lineup, to do they can just ask for an optimization on their lineup or provide a tactic too\
                They might ask questions related what to who they should have for defenders, midfielders or forwards, or just give the tactic their team plays in\
                When a tactic is provided it should follow one of these formats 433, 4-3-3, 4/3/3, try to be creative, a tactic may also not be provided and a user can also ask for a specific number of players for a certain position"

response = synthetic_data_chain.invoke({"k": k, "user_intentions": user_intentions, "target_task_intention": intention, "target_task_intention_description": description})

optimize_lineup_messages = []
for message in response.messages:
    optimize_lineup_messages.append({"Intention":intention, "Message":message.message})

In [54]:
optimize_lineup_messages[:5]

[{'Intention': 'optimize_lineup',
  'Message': 'Can you help me optimize my lineup for a 4-3-3 formation?'},
 {'Intention': 'optimize_lineup',
  'Message': 'What players should I choose for my midfield in a 4/3/3?'},
 {'Intention': 'optimize_lineup',
  'Message': 'I need suggestions to optimize my lineup for this weekend.'},
 {'Intention': 'optimize_lineup',
  'Message': 'How can I improve my defense in a 4-3-3 setup?'},
 {'Intention': 'optimize_lineup',
  'Message': 'Which forwards should I prioritize for my starting eleven?'}]

In [55]:
add_messages(optimize_lineup_messages, file_name)

# Intention 3: Putting Players in Starting11

In [56]:
intention = "putting_players_in_starting11"

description = "The user wants to make a change in their starting eleven. To do so they must define wether they want a player into his starting team of out of his starting team\
                They might make statements related to taking players out of their starting eleven or putting a player in their starting eleven"

response = synthetic_data_chain.invoke({"k": k, "user_intentions": user_intentions, "target_task_intention": intention, "target_task_intention_description": description})

putting_players_in_starting11_messages = []
for message in response.messages:
    putting_players_in_starting11_messages.append({"Intention":intention, "Message":message.message})

In [57]:
putting_players_in_starting11_messages[:5]

[{'Intention': 'putting_players_in_starting11',
  'Message': 'I want to put Salah in my starting eleven.'},
 {'Intention': 'putting_players_in_starting11',
  'Message': 'Can I take out Kane from my starting lineup?'},
 {'Intention': 'putting_players_in_starting11',
  'Message': 'Please add Bruno Fernandes to my starting team.'},
 {'Intention': 'putting_players_in_starting11',
  'Message': 'I need to remove Son from my starting eleven.'},
 {'Intention': 'putting_players_in_starting11',
  'Message': "Let's put Haaland in the starting lineup."}]

In [58]:
add_messages(putting_players_in_starting11_messages, file_name)

# Intention 4: Know Information about Football Stats

In [59]:
intention = "know_information_about_stats"

description = "The user wants to know something about one common football stat or something related to a stat and fantasy football\
                They might make questions either related to stats themselves like what is a goal or about something relating stats and fantasy football, like the amount of fantasy points for a goal"
                
response = synthetic_data_chain.invoke({"k": k, "user_intentions": user_intentions, "target_task_intention": intention, "target_task_intention_description": description})

know_information_about_stats_messages = []
for message in response.messages:
    know_information_about_stats_messages.append({"Intention":intention, "Message":message.message})


In [60]:
know_information_about_stats_messages[:5]

[{'Intention': 'know_information_about_stats',
  'Message': 'What does a goal count for in fantasy points?'},
 {'Intention': 'know_information_about_stats',
  'Message': 'How many points do assists earn in fantasy football?'},
 {'Intention': 'know_information_about_stats',
  'Message': 'Can you explain how clean sheets affect player stats?'},
 {'Intention': 'know_information_about_stats',
  'Message': 'What is the point value for a penalty save?'},
 {'Intention': 'know_information_about_stats',
  'Message': 'How are yellow cards factored into player scoring?'}]

In [61]:
add_messages(know_information_about_stats_messages, file_name)

# Intention 5: Recommend Player

In [62]:
intention = "recommend_players"

description = "The user wants to get players recommended to him. To do so he may ask for certain position, choose the number of players he want recommended, some players they want to exclude, or even set a budget\
                They might make statements containing the position, number, budget, players they want to exclude, or neither and just ask for recommendations."
                
response = synthetic_data_chain.invoke({"k": k, "user_intentions": user_intentions, "target_task_intention": intention, "target_task_intention_description": description})

recommend_players_messages = []
for message in response.messages:
    recommend_players_messages.append({"Intention":intention, "Message":message.message})


In [63]:
recommend_players_messages[:5]

[{'Intention': 'recommend_players',
  'Message': 'Can you recommend some midfielders for my team?'},
 {'Intention': 'recommend_players',
  'Message': 'I need two forwards. Any suggestions?'},
 {'Intention': 'recommend_players',
  'Message': 'Please recommend players under 8 million.'},
 {'Intention': 'recommend_players',
  'Message': 'Who are the best defenders to pick this week?'},
 {'Intention': 'recommend_players',
  'Message': 'Suggest me three players to consider for my lineup.'}]

In [64]:
add_messages(recommend_players_messages, file_name)

# Intention 6: Transfer Players

In [66]:
intention = "transfer_players"

description = "The user wants to take a player out of his squad and replace him with another os simply add a new player, if the user wants to replace a player in his squad he need to tell both names and if he just wants to add a new one he just has to tell one name\
                They might make statements either taking a player out and bringing another in or just adding a player to the team"
                
response = synthetic_data_chain.invoke({"k": k, "user_intentions": user_intentions, "target_task_intention": intention, "target_task_intention_description": description})

transfer_players_messages = []
for message in response.messages:
    transfer_players_messages.append({"Intention":intention, "Message":message.message})


In [67]:
transfer_players_messages[:5]

[{'Intention': 'transfer_players',
  'Message': 'I want to transfer out Salah and bring in Son.'},
 {'Intention': 'transfer_players', 'Message': 'Can I add Haaland to my team?'},
 {'Intention': 'transfer_players',
  'Message': 'Please replace Bruno with KDB.'},
 {'Intention': 'transfer_players',
  'Message': 'I’d like to transfer out Kane for Nunez.'},
 {'Intention': 'transfer_players', 'Message': 'Add Rashford to my squad.'}]

In [68]:
add_messages(transfer_players_messages, file_name)

# Intention 7: View or Update Team Points

In [69]:
intention = "view_or_update_team_points"

description = "The user wants to either update or view his team points, if he wants to just view all he needs to do is ask , but if he wants to update he will need give an integer from 0 to 100 that is his new team points\
                They might make statements either saying they want to view or to update their points"

response = synthetic_data_chain.invoke({"k": k, "user_intentions": user_intentions, "target_task_intention": intention, "target_task_intention_description": description})

view_or_update_team_points_messages = []
for message in response.messages:
    view_or_update_team_points_messages.append({"Intention":intention, "Message":message.message})

In [70]:
view_or_update_team_points_messages[:5]

[{'Intention': 'view_or_update_team_points',
  'Message': 'Can you show me my current team points?'},
 {'Intention': 'view_or_update_team_points',
  'Message': 'I want to check how many points my team has.'},
 {'Intention': 'view_or_update_team_points',
  'Message': 'What are my team points right now?'},
 {'Intention': 'view_or_update_team_points',
  'Message': 'Please display my team points for me.'},
 {'Intention': 'view_or_update_team_points',
  'Message': "I'd like to see my total team points."}]

In [71]:
add_messages(view_or_update_team_points_messages, file_name)

# Intention 8: View players stats

In [72]:
intention = "view_players_stats"

description = "The user wants to view the stats for a certain player he can ask for what stats he wants (goal, assists, price evolution, form rank, and more) or simply say nothing and get them all\
                They will make statements containing the name of the player and asking for information."

response = synthetic_data_chain.invoke({"k": k, "user_intentions": user_intentions, "target_task_intention": intention, "target_task_intention_description": description})

view_players_stats_points_messages = []
for message in response.messages:
    view_players_stats_points_messages.append({"Intention":intention, "Message":message.message})

In [73]:
view_players_stats_points_messages[:5]

[{'Intention': 'view_players_stats',
  'Message': 'Can you show me the stats for Harry Kane?'},
 {'Intention': 'view_players_stats',
  'Message': "What are Mohamed Salah's goals and assists this season?"},
 {'Intention': 'view_players_stats',
  'Message': "I'd like to see the price evolution for Kevin De Bruyne."},
 {'Intention': 'view_players_stats',
  'Message': "Please provide me with Son Heung-min's current form rank."},
 {'Intention': 'view_players_stats',
  'Message': 'What are the latest stats for Bruno Fernandes?'}]

In [74]:
add_messages(view_players_stats_points_messages, file_name)

# Intention 9: Chit chat about the company

In [75]:
intention = "chit_chat_about_company"

description = "The user makes some chit chat asking about the company\
                They will make statements and questions about the company. The company is named Ball IQ\
                use words like the name, company and synonyms "

response = synthetic_data_chain.invoke({"k": 25, "user_intentions": user_intentions, "target_task_intention": intention, "target_task_intention_description": description})

chit_chat_about_company_messages = []
for message in response.messages:
    chit_chat_about_company_messages.append({"Intention":intention, "Message":message.message})

In [76]:
chit_chat_about_company_messages[:5]

[{'Intention': 'chit_chat_about_company',
  'Message': 'What inspired the name Ball IQ for your company?'},
 {'Intention': 'chit_chat_about_company',
  'Message': 'How did Ball IQ get started in the fantasy football space?'},
 {'Intention': 'chit_chat_about_company',
  'Message': 'Can you share more about the mission of Ball IQ?'},
 {'Intention': 'chit_chat_about_company',
  'Message': 'What sets Ball IQ apart from other fantasy football platforms?'},
 {'Intention': 'chit_chat_about_company',
  'Message': 'How does Ball IQ ensure user satisfaction and engagement?'}]

In [77]:
add_messages(chit_chat_about_company_messages, file_name)

# No Intention: None

In [78]:
system_prompt = """
You are tasked with generating synthetic user messages.

The user intentions are:
{user_intentions}

Your task is to create {k} distinct messages completely unrelated to the available user intentions.
These messages should be generic and not related to any specific task or intention.
The user is engaging in casual conversation.
The user might ask general questions, share opinions, or express emotions. 
The user might also ask for totaly none related questions to the platform. 
The user might ask general questions, share opinions, or express emotions.

Follow these guidelines:
1. Focus exclusively on not being related to any of the user intentions.
2. Each message should be between 5 and 20 words.
3. Avoid including any details or references to other user intentions.
4. Ensure the messages sound natural and typical of user queries for the given intention.
5. Follow the provided format strictly to maintain consistency.

Message format:
{format_instructions}
"""

In [79]:
prompt = PromptTemplate(
    template=system_prompt,
    input_variables=["k", "user_intentions"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

synthetic_data_chain = prompt | llm | output_parser

In [80]:
response = synthetic_data_chain.invoke({"k": 25, "user_intentions": user_intentions})

none_related_messages = []

for message in response.messages:
    none_related_messages.append({"Intention":"None", "Message":message.message})

In [81]:
none_related_messages[:5]

[{'Intention': 'None', 'Message': "What's your favorite movie of all time?"},
 {'Intention': 'None',
  'Message': "I love the weather today, it's so nice outside!"},
 {'Intention': 'None', 'Message': 'Have you read any good books lately?'},
 {'Intention': 'None',
  'Message': 'What do you think about the latest tech trends?'},
 {'Intention': 'None',
  'Message': 'I tried a new recipe yesterday, it turned out great!'}]

In [82]:
add_messages(none_related_messages, file_name)